# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     | Iaabela Torres -Septien Uribe |
| **Fecha**      | 15 de septiembre 2025  |
| **Expediente** | 730667  |

## Validación

In [17]:
import pandas as pd

In [18]:
import numpy as np

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [32]:
data = pd.read_excel("Motor Trend Car Road Tests.xlsx")
data = data.drop(columns = ['model'])
data.tail()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
27,30.4,4,95.1,113,3.77,1.513,16.9,1,1,5,2
28,15.8,8,351.0,264,4.22,3.170,14.5,0,1,5,4
29,19.7,6,145.0,175,3.62,2.770,15.5,0,1,5,6
30,15.0,8,301.0,335,3.54,3.570,14.6,0,1,5,8
31,21.4,4,121.0,109,4.11,2.780,18.6,1,1,4,2


In [33]:
x = data.drop(columns = ['mpg'])
y = data['mpg']

In [34]:
mses = []

for i in range(32):
  x_train = x.drop(i).to_numpy()
  x_test = x.iloc[i].to_numpy().reshape(1, -1)
  y_train = y.drop(i).to_numpy()
  y_test = y.iloc[i]

  scaler = StandardScaler()
  x_train = scaler.fit_transform(x_train)
  x_test = scaler.transform(x_test)

  lr = LinearRegression()
  lr.fit(x_train, y_train)

  y_pred = lr.predict(x_test)

  mse = mean_squared_error([y_test], y_pred)
  mses.append(mse)

print("MSE promedio:", np.mean(mses))
print("Desviación estándar:", np.std(mses))



MSE promedio: 12.181558006901941
Desviación estándar: 17.067399871888586


Interpreta.

El resultado muestra que el modelo tiene un error cuadrático medio (MSE) promedio de aproximadamente 12.18. Esto indica un error moderado. Sin embargo, la desviación estándar es de 17.07,Esto es bastante alto por lo que el modelo no será el más optimo o mejor.



## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [80]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [82]:
import numpy as np

np.random.seed(42)

indices = np.arange(len(X))
np.random.shuffle(indices)

Xs = X[indices]
ys = y[indices]

lista1 = indices[1:2064]
lista2 = indices[2064:4128]


In [100]:
tam_seccion = len(indices) // num_secciones

folds = []
for i in range(10)
    inicio = i * tam_seccion
    fin = (i + 1) * tam_seccion
    listas.append(indices[inicio:fin])


SyntaxError: invalid syntax (2215274097.py, line 3)

In [92]:
lista1 = indices[1:2064]
lista2 = indices[2064:4128]


numpy.ndarray

In [96]:
pd.DataFrame(Xs)

,0,1,2,3,4,5,6,7
0,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01
1,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46
2,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44
3,5.7376,17.0,6.163636,1.020202,1705.0,3.444444,34.28,-118.72
4,3.7250,34.0,5.492991,1.028037,1063.0,2.483645,36.62,-121.93
...,...,...,...,...,...,...,...,...
20635,6.3700,35.0,6.129032,0.926267,658.0,3.032258,33.78,-117.96
20636,3.0500,33.0,6.868597,1.269488,1753.0,3.904232,34.02,-117.43
20637,2.9344,36.0,3.986717,1.079696,1756.0,3.332068,34.03,-118.38
20638,5.7192,15.0,6.395349,1.067979,1777.0,3.178891,37.58,-121.96


In [98]:
# Hacer un fold
(20640/10)*2

4128.0

In [110]:
import numpy as np

np.random.seed(42)

indices = np.arange(len(X))
np.random.shuffle(indices)

Xs = X[indices]
ys = y[indices]

lista1 = indices[1:2064]
lista2 = indices[2064:4128]


In [112]:

tam_seccion = len(indices) // 10

folds = []
for i in range(10):
    inicio = i * tam_seccion
    fin = (i + 1) * tam_seccion
    folds.append(indices[inicio:fin])


In [122]:

mses = []
for i in range(10):
    inicio = i * tam_seccion
    fin = (i + 1) * tam_seccion
    
    test_idx = indices[inicio:fin]
    train_idx = np.concatenate((indices[:inicio], indices[fin:]))
    
    x_train = Xs[train_idx]
    x_test = Xs[test_idx]
    y_train = ys[train_idx]
    y_test = ys[test_idx]

    lr = LinearRegression()
    lr.fit(x_train, y_train)

    y_pred = lr.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    mses.append(mse)

print("MSE promedio:", np.mean(mses))
print("Desviación estándar:", np.std(mses))

MSE promedio: 0.5270659954870702
Desviación estándar: 0.022266703148739637


El MSE es bastante bajo por lo tanto el error es muy bajo por lo tanto la regresión lineal es muy buena. EL error es de 0.5270 aproximadamente 0.53 por lo que se puede decir que es bastante bueno además de que la desviación estandar es muy baja siendo 0.022

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3